## Config

In [1]:
### Configuration
# Dependencies
import os
import pandas as pd
import numpy as np
from summerplaylist import *

# Environment variables
CLIENTID = os.environ.get('CLIENTID')
CLIENTSECRET = os.environ.get('CLIENTSECRET')
REDIRECT_URI = 'https://example.com/callback'

# Filepaths
artistsKnownPath = os.path.join('data', 'artistsKnown.csv')

# Set max age for new artist songs
maxAge = 5

## Log In

In [3]:
# Authenticate to Spotify
credentials = authenticateToSpotify(CLIENTID, CLIENTSECRET, REDIRECT_URI)

Opening browser for Spotify login...


## Collect new artists

In [4]:
# This block can be commented out once it has been run once. It does not update each year.

# # Collect names of new artists from Music Festival Wizard
# artistsNew = fetchMusicFestivalWizard()
# # Filter out known artists who we have already listened to
# artistsNew = artistsNew.loc[~artistsNew['artistName'].isin(artistsKnown['artistName'])]
# artistsNew = artistsNew.head()
# # Search spotify for artistIDs
# artistsNew['artistID'] = artistsNew.apply(findArtistIDs, credentials=credentials, axis=1)
# # Filter out if artistID was known
# artistsNew = artistsNew.loc[~artistsNew['artistID'].isin(artistsKnown['artistID'])]
# artistsNew

In [5]:
# Print the playlist IDs so user can select an id to add to "New Artists to Explore"
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta
from collections import Counter
import pandas as pd
import numpy as np
from requests_html import HTMLSession
import tekore as tk

spotify, userID, user_token = credentials
with spotify.token_as(user_token):
    # Check if playlist already exists; if not create a new one
    userPlaylists = pd.DataFrame([{'id':x.id, 'name':x.name} for x in spotify.playlists(userID, limit=40).items])

userPlaylists

,id,name
0,7EUJFYC05KklRfKE3FQkcR,Eurovision24
1,1KPjybavao8KU78UZYhx30,Endless Summer 2022
2,4BSj7IwrHLRpZeVNodMh5Z,Coachella 2024 Official Playlist
3,4aTeCEuSx2YK1VtJhpRr7V,Summer 2022
4,2linyGNYT2pFx6Mb6TTjVb,All Summers
5,33jMFYNrURbRExrzuqMfJ6,Endless Summer 2021
6,0Yj1Gqn15HdcDNcV5XJqVm,New Artists to Explore Further
7,64ehl2sLfLPIF8qnsZixlv,Summer 2021
8,7ecbDSTLLua1FsHah5M061,SFW Summer 2023
9,5xpbKk27Sjs0z5Uhs7DJsW,Endless Summer 2020


## Process new artists

In [6]:
# Read in known artists
artistsKnown = pd.read_csv(artistsKnownPath)

In [31]:
# Collect artists from an existing favorited playlist
playlistToFetch = '4BSj7IwrHLRpZeVNodMh5Z' #getPlaylistID(credentials, 'New Artists to Explore')
artistsNew = fetchTracksFromPlaylist(credentials, playlistToFetch)
artistsNew = artistsNew[['artistName', 'artistID']].drop_duplicates('artistID', ignore_index=True).reset_index(drop=True)
# Filter out if artistID was known
artistsNew = artistsNew.loc[~artistsNew['artistID'].isin(artistsKnown['artistID'])]
artistsNew

,artistName,artistID
11,Ice Spice,3LZZPxNDGDFVSIPqf4JuEf
20,Quevedo,52iwsT98xCoGgiGntTiR7K
21,Sublime,0EdvGhlC1FkGItLOWQzG4J
23,Carin Leon,66ihevNkSYNzRAl44dx6jJ
27,Chris Avantgarde,715OI7hiv58daVlEDXM47U
...,...,...
262,Maori,2GjC0P8uCItsOxEYXtm7kv
264,The Dayshift,2U72zkP4NU5KqCSWuH3lyP
267,Romy,3X2DdnmoANw8Rg8luHyZQb
268,Sadye,73L9ebWPXM8bSZq6EnydUf


## Get top songs

In [29]:
# Get their top three songs in last 5 years
songURIs = fetchTopSongs(credentials, artistsNew['artistID'], maxAge=maxAge)
# How many songs are in this round?
print(f"There are {len(songURIs)} songs about to be added")

There are 45 songs about to be added


## Put songs in playlist

In [30]:
# Warning: This will make a new playlist if it does not already exist
playlistID = getPlaylistID(credentials, 'Summer - Brand New Artists')
# Add songs to the playlist
addToPlaylist(credentials, songURIs, playlistID)

## Update the list of known artists

In [32]:
# Append artistsNew to artistsKnown.csv 
allArtists = pd.concat([artistsKnown, artistsNew], axis=0, ignore_index=True)
allArtists.to_csv(artistsKnownPath, index=False)

## Create playlist to drop good songs into

In [21]:
# Cull exploreArtists into ArtistHasPotential playlist
winnersPlaylist = getPlaylistID(credentials, 'Summer - New Artists to Explore Further')
print(f'Now drop songs you like into this playlist: {winnersPlaylist}')

Now drop songs you like into this playlist: 0Yj1Gqn15HdcDNcV5XJqVm
